In [1]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
import pickle
np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [62]:
head = single_head.SingleHead.read_from_file(1,1)
head.full_filter()
head.get_filtered_image()

depth filter done.
dangling removal done
color filter done.
